<a href="https://colab.research.google.com/github/a150730/NLP-projects/blob/main/Feature_extraction_and_Classification_using_LR%2CNB_and_RF_with_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import module

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

### text data

In [3]:
texts = ["good movie","not a good movie","did not like","i like it","good one"]

### initialize tdidf

In [4]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))



In [5]:
features = tfidf.fit_transform(texts)

### view result

In [6]:
pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())

,good movie,like,movie,not
0,0.707107,0.000000,0.707107,0.000000
1,0.577350,0.000000,0.577350,0.577350
2,0.000000,0.707107,0.000000,0.707107
3,0.000000,1.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000


In [87]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/1_KG_4-26.csv')

In [96]:
dataRaw = dataset.dropna()


In [97]:
dataRaw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Raw        89 non-null     object
 1   CD-Malaya  89 non-null     object
 2   CD-RE      89 non-null     object
 3   Polarity   89 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 3.5+ KB


In [100]:
dataRaw.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/for feature extraction/dfkg_1.csv', index = False, header=True)

In [101]:
dfkg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/for feature extraction/dfkg_1.csv')

In [102]:
datacdre = dfkg['CD-RE']
dataPolarity = dfkg['Polarity']

In [105]:
dataPolarity

0     1
1    -1
2    -1
3    -1
4    -1
     ..
84   -1
85    0
86    0
87    0
88   -1
Name: Polarity, Length: 89, dtype: int64

In [106]:
text2= list(datacdre)

In [107]:
stopWord= ["and","be","in","at","di","ini","sini","is","kan","ke","ku","yg","yang","is","la","lah","lg","lagi","mcm","ni","rm","st","up","us","to","the"]

In [108]:
tfidf2 = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2), stop_words=stopWord)

In [109]:
feature2 = tfidf2.fit_transform(text2)

In [110]:
dfkgVec = pd.DataFrame(feature2.todense(), columns=tfidf2.get_feature_names())

In [111]:
dfkgVec.style

,ada,adalah,allah,amp,awam,bagi,balik,balik pinjaman,bantuan,bantuan khas,banyak,bawah,bayaran,bayaran balik,being,beri,berjaya,berjumlah,bertujuan,betul,boleh,buat,bukan,bulan,bulan ramadhan,covid,cukai,dah,dalam,dan,disediakan,doa,don,duit,ekonomi,everytime,fazura,function,gagal,geran,government,hari,hati,have,ingat,inisiatif,isu,judi,juga,just,juta,kadar,kami,kata,kena,kepada,kerajaan,kesihatan,khas,khas permai,kita,kompaun,kwsp,lain,let,let this,living,living malaysia,makin,malaysia,mati,menampung,menang,menteri,moratorium,nak,need,negara,no,one,orang,org,otak,our,pada,pakej,pakej bantuan,pelan,pembelian,penyamun,perasuah,perbelanjaan,permai,pinjaman,please,prihatin,program,program subsidi,rakyat,rakyat malaysia,ramadhan,raya,salah,say,sebab,sebanyak,sehingga,sekarang,semasa,semasa tempoh,semua,serta,siapa,subsidi,subsidi upah,sudah,tahu,tak,tak function,tapi,tau,tempoh,terima,terjejas,this,untuk,upah,warga,we,week,what,where,you,you don,zalim
0,0.311202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.653006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.690461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.382333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.424140,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.401131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.453802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.401131,0.000000,0.000000,0.382333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.491191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.491191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.525543,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [112]:
dataToProcess = pd.concat([dfkgVec, dataPolarity], axis=1, ignore_index=False)


In [113]:
dataToProcess.style


,ada,adalah,allah,amp,awam,bagi,balik,balik pinjaman,bantuan,bantuan khas,banyak,bawah,bayaran,bayaran balik,being,beri,berjaya,berjumlah,bertujuan,betul,boleh,buat,bukan,bulan,bulan ramadhan,covid,cukai,dah,dalam,dan,disediakan,doa,don,duit,ekonomi,everytime,fazura,function,gagal,geran,government,hari,hati,have,ingat,inisiatif,isu,judi,juga,just,juta,kadar,kami,kata,kena,kepada,kerajaan,kesihatan,khas,khas permai,kita,kompaun,kwsp,lain,let,let this,living,living malaysia,makin,malaysia,mati,menampung,menang,menteri,moratorium,nak,need,negara,no,one,orang,org,otak,our,pada,pakej,pakej bantuan,pelan,pembelian,penyamun,perasuah,perbelanjaan,permai,pinjaman,please,prihatin,program,program subsidi,rakyat,rakyat malaysia,ramadhan,raya,salah,say,sebab,sebanyak,sehingga,sekarang,semasa,semasa tempoh,semua,serta,siapa,subsidi,subsidi upah,sudah,tahu,tak,tak function,tapi,tau,tempoh,terima,terjejas,this,untuk,upah,warga,we,week,what,where,you,you don,zalim,Polarity
0,0.311202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.653006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.690461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
1,0.382333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.424140,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.401131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.453802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.401131,0.000000,0.000000,0.382333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1
2,0.000000,0.000000,0.000000,0.491191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.491191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.525543,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

### train test split using sklearn model selection

In [128]:
from sklearn.model_selection import train_test_split

In [129]:
print(dataToProcess.shape )

(89, 136)


In [130]:
x = dfkgVec
y = dataPolarity

In [131]:
feature_train, feature_test, target_train, target_test = train_test_split(x,y, test_size = 0.33, random_state = 42)

In [132]:
print(feature_train.shape)
print(feature_test.shape)
print(target_train.shape)
print(target_test.shape)

(59, 135)
(30, 135)
(59,)
(30,)


### import model

In [151]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

### initialize classifiesr model (Logistic Regression)

In [134]:
model = LogisticRegression(class_weight="balanced")


In [135]:
model.fit(feature_train,target_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [142]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [139]:
target_pred = model.predict(feature_test)
target_pred

array([ 1,  1,  1,  0,  1, -1,  0,  0,  0, -1,  0,  0,  0,  0, -1,  1,  0,
        0, -1,  1,  1,  1,  1,  0, -1,  0,  0, -1, -1,  1])

#### evaluate our LR model

In [143]:
#f1score = f1_score(target_test, target_pred)
#evaluate our logistic regression model accuracy
print(accuracy_score(target_test,target_pred))

0.6666666666666666


### initialize classifiesr model (naive Bayes)

In [147]:
#initialize naive Bayes classifier
gnb = GaussianNB()

In [148]:
#Train classifier
gnbmodel = gnb.fit(feature_train,target_train)

In [149]:
#Make prediction using our gnb
gnb_target_pred = gnbmodel.predict(feature_test)
print(gnb_target_pred)

[ 1  1 -1  0  1 -1  0  0  0 -1  0  1 -1  0 -1  1  0  0 -1  1 -1  1  1  0
 -1 -1  0 -1 -1  1]


#### Evaluate our NB model

In [150]:
#evaluate our gnb model accuracy
print(accuracy_score(target_test,gnb_target_pred))

0.6333333333333333


### initialize classifiesr model (Random Forest )

In [152]:
rf = RandomForestClassifier()
#Train classifier
rfmodel = rf.fit(feature_train,target_train)

In [153]:
#Make prediction using our rf model
rf_target_pred = rfmodel.predict(feature_test)
print(rf_target_pred)

[ 0  1  0  0  1 -1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  1  1  1  0
  0  0  0  0  0  0]


#### Evaluate our model ( Random Forest)

In [154]:
#evaluate our random forest model accuracy
print(accuracy_score(target_test,rf_target_pred))

0.43333333333333335
